In [45]:
import pandas as pd
import json

In [46]:
def refactor_values(data):
    new_data = {}
    for key, value in data.items():
            if value != '':
                new_data[key] = value
            if isinstance(value, str) and '<' in value:
                new_value = value.split('<')[1]
                new_data[key] = int(new_value)
    return new_data

In [47]:
file = open('../data/raw/data.json', 'r', encoding='utf-8')
raw = json.load(file)
file.close()

df = pd.DataFrame()

for i in range(len(raw)):
    jdata = json.loads(raw[i]['json'])
    station = jdata['arsopodatki']['postaja']
    for i in range(len(station)):
        data = station[i]
        data = refactor_values(data)
        df = pd.concat([df, pd.json_normalize(data)])  
        
#df

In [48]:
print(df.isnull().sum())

nadm_visina          0
datum_od             0
merilno_mesto        0
benzen           59707
ge_sirina         3073
sifra                0
datum_do             0
ge_dolzina        3073
pm2.5            10046
o3               35265
pm10             10046
co               61793
no2              37066
so2              56683
dtype: int64


In [49]:
cat_cols = df.select_dtypes(include=['object']).columns
print(cat_cols)
num_cols = df.select_dtypes(include=['number']).columns
print(num_cols)

Index(['datum_od', 'merilno_mesto', 'sifra', 'datum_do'], dtype='object')
Index(['nadm_visina', 'benzen', 'ge_sirina', 'ge_dolzina', 'pm2.5', 'o3',
       'pm10', 'co', 'no2', 'so2'],
      dtype='object')


In [50]:
for col in num_cols:
    df[col].fillna((df[col].mean()), inplace=True)

df_location = pd.get_dummies(df['merilno_mesto'])
df = pd.concat([df, df_location], axis=1).reindex(df.index)
df.pop('merilno_mesto')
df.pop('sifra')
df.pop('datum_od')
df.pop('datum_do')

df

,nadm_visina,benzen,ge_sirina,ge_dolzina,pm2.5,o3,pm10,co,no2,so2,...,MB Vrbanski,MS Cankarjeva,MS Rakičan,NG Grčna,Novo mesto,Otlica,Ptuj,Rečica v I.Bistrici,Trbovlje,Zagorje
0,299,1.000000,46.065851,14.517454,35.610733,51.407339,42.277626,0.452701,24.180617,2.210446,...,0,0,0,0,0,0,0,0,0,0
0,305,1.093431,46.072399,14.491849,35.610733,51.407339,42.277626,0.452701,24.180617,2.210446,...,0,0,0,0,0,0,0,0,0,0
0,293,1.093431,46.037791,14.494001,35.610733,51.407339,42.277626,0.452701,24.180617,2.210446,...,0,0,0,0,0,0,0,0,0,0
0,388,1.093431,46.242115,14.366963,35.610733,51.407339,42.277626,0.452701,24.180617,2.210446,...,0,0,0,0,0,0,0,0,0,0
0,270,1.500000,46.559202,15.656191,35.610733,51.407339,42.277626,0.452701,24.180617,2.210446,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,241,1.093431,46.131416,15.000906,32.000000,84.000000,34.000000,0.452701,8.000000,3.000000,...,0,0,0,0,0,0,0,0,0,1
0,290,1.093431,46.144213,15.088097,28.000000,51.407339,29.000000,0.452701,24.180617,2.210446,...,0,0,0,0,0,0,0,0,0,0
0,220,1.093431,45.802080,15.182060,33.000000,72.000000,35.000000,0.452701,6.000000,2.210446,...,0,0,0,0,1,0,0,0,0,0
0,540,0.000000,45.561344,14.862816,12.000000,88.000000,14.000000,0.452701,24.180617,1.000000,...,0,0,0,0,0,0,0,0,0,0


In [51]:
df.to_csv('../data/processed/data.csv', index=False)